In [1]:
import os
import numpy as np
from glob import glob
from tqdm import tqdm
import pandas as pd
from lungmask import mask as masker
pids = os.listdir('/data/datasets/LIDC_IDRI/dicoms')
from matplotlib import pyplot as plt

In [2]:
pady,padx = 15,5 
todel = ['LIDC-IDRI-0634-0-0', 'LIDC-IDRI-0545-1-6'] # these two slices have no lung area

base = '/data/liumingzhou/CounterAlign_output/preprocess/multiple_slices/raw'
raw = pd.read_csv(base+'.csv').set_index('filename')
pids = os.listdir('/data/datasets/LIDC_IDRI/dicoms')

#### segmentation with lungmask api

In [3]:
def backtoHU(image,inter=0,slop=1):
    '''
    0~1 np.array back to HU valued; intercept/slope from dicom info
    '''
    MIN_BOUND = -1000.0
    MAX_BOUND = 400.0
    image = image*(MAX_BOUND-MIN_BOUND) + MIN_BOUND
    image = (image-inter)/slop
    return image.astype(np.float64)

#### check

In [4]:
def segmask2boundbox(mask,pady,padx):
    '''
    we use numpy coordinate system; np.array[idy,idx]
    '''
    if len(mask.shape)==3:
        mask = mask[0,:]
    shape = mask.shape
    ycoords, xcoords = np.where(mask!=0)
    assert len(ycoords)>0
    ymin, ymax, xmin, xmax = ycoords.min(), ycoords.max(), xcoords.min(), xcoords.max()

    ymin = max(ymin-pady,0)
    ymax = min(ymax+pady,shape[0])
    xmin = max(xmin-padx,0)
    xmax = min(xmax+padx,shape[1])
    return [xmin,ymin,xmax,ymax]

In [5]:
# each image has a segmask
for file in os.listdir(base):
    if 'npy' in file and 'lungmask' not in file:
        where_mask = os.path.join(base,file.split('.npy')[0]+'-lungmask.npy')
        assert os.path.exists(where_mask), where

In [6]:
# each segmask is valid
errorfiles = list()
for file in raw.index:
    mask = np.load(os.path.join(base,file+'-lungmask.npy'))
    try:
        box = segmask2boundbox(mask,pady,padx)
    except AssertionError:
        if file not in todel:
            errorfiles.append(file)
            
if len(errorfiles)>0:
    print('Find {} images with invalid mask. Re-computing ...'.format(len(errorfiles)))
    for file in tqdm(errorfiles):
        img = np.load(os.path.join(base,file+'.npy'))
        mask = masker.apply(backtoHU(img)[np.newaxis,:], batch_size=1)[0,:]   
        savename = os.path.join(base,file+'-lungmask.npy')
        np.save(savename,mask)

#### save lung box to raw.csv

In [11]:
def filename2boundbox(filename):
    '''
    given lidc-idri-pid-nid-sliceid, return [xmin,ymin,xmax,ymax] as bounding box of the lung
    '''
    if filename in todel:
        box = [0,0,0,0]
    else:
        mask = np.load(os.path.join(base,filename+'-lungmask.npy'))
        box = segmask2boundbox(mask,pady,padx)
    return box

In [12]:
boxs = list(map(filename2boundbox,list(raw.index)))
raw['lungbox'] = boxs
raw.to_csv(base+'.csv')

#### visualization